In [ ]:
!pip install langdetect
!pip install langid
import requests
from bs4 import BeautifulSoup
import re
import json
from langdetect import detect
import uuid
import os
from pathlib import Path
import langid



In [ ]:
import requests
from bs4 import BeautifulSoup
import json
from langdetect import detect
import langid

class AmharicTextScraper:
    def __init__(self, url, jsonl_path,filename="webpage_amharic_text.jsonl"):
        self.url = url
        self.jsonl_path = jsonl_path
        self.amharic_text = []
        self.page_content = None
        self.jsonl_path = jsonl_path
        self.file_name=filename

    def fetch_page(self):
        # headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(self.url, headers=headers)

        if response.status_code != 200:
            print(f"Failed to access {self.url}, Status Code: {response.status_code}")
            return self

        if langid.classify(response.text)[0] != "am":
            print(f"The page at {self.url} is not detected as Amharic.")
            return self

        self.page_content = response.text
        return self

    def extract_amharic_text(self):
        if not self.page_content:
            print("No page content to process")
            return self

        soup = BeautifulSoup(self.page_content, "html.parser")
        paragraphs = soup.find_all("p")

        for p in paragraphs:
            text = p.get_text(strip=True)
            if not text:
                continue

            try:
                lang, confidence = langid.classify(text)
                if (lang == "am" and confidence > -50) or self.is_probably_amharic(text):
                    self.amharic_text.append(text)
            except Exception as e:
                print("Language detection failed:", e)
                pass

        return self

    def is_probably_amharic(self, text):
        # Checks if any character is in the Amharic Unicode block
        return any('\u1200' <= char <= '\u137F' for char in text)

    def save_to_jsonl(self):
        path = os.path.join(self.jsonl_path, self.file_name)
         # Create the directory if it doesn't exist
        os.makedirs(self.jsonl_path, exist_ok=True)

        with open(path, 'a', encoding='utf-8') as f:
            for text in self.amharic_text:
                json.dump({'url': self.url, 'text': text}, f, ensure_ascii=False)
                f.write("\n")
        print(f"Extracted Amharic text saved to {self.jsonl_path}")
        return self

    def run(self):
        return self.fetch_page().extract_amharic_text().save_to_jsonl()


In [ ]:
page_url = "https://www.fanamc.com/archives/288627"
text=AmharicTextScraper(page_url,"drive/MyDrive/scraped_data","amharic.jsonl").run()


Extracted Amharic text saved to drive/MyDrive/scraped_data


In [ ]:
!pip install pymupdf
import pymupdf as fitz
class AmharicPDFFetcher:
    def __init__(self, pdf_url, save_dir, file_name="amharic_from_pdf.jsonl"):
        self.pdf_url = pdf_url
        self.save_dir = Path(save_dir)
        self.file_name = file_name
        self.amharic_texts = []

        # Create the save directory if it doesn't exist
        self.save_dir.mkdir(parents=True, exist_ok=True)

    def download_pdf(self):
        response = requests.get(self.pdf_url)
        if response.status_code != 200:
            raise Exception(f"Failed to download PDF. Status code: {response.status_code}")

        self.pdf_path = self.save_dir / "temp.pdf"
        with open(self.pdf_path, "wb") as f:
            f.write(response.content)
        print(f"✅ PDF downloaded to {self.pdf_path}")
        return self

    def extract_amharic_text(self):
      amharic_re = re.compile(r'[\u1200-\u137F]+')
      doc = fitz.open(self.pdf_path)
      for page in doc:
          text = page.get_text()
          for line in text.splitlines():
              line = line.strip()
              if line and amharic_re.search(line):
                  self.amharic_texts.append(line)
      doc.close()
      print(f"✅ Extracted {len(self.amharic_texts)} Amharic lines.")
      return self

    def save_to_jsonl(self):
        output_path = self.save_dir / self.file_name
        with open(output_path, "a", encoding="utf-8") as f:
            for line in self.amharic_texts:
                json.dump({'url': self.pdf_url, 'text': line}, f, ensure_ascii=False)
                f.write("\n")
        print(f"✅ Saved to {output_path}")
        return self

    def run(self):
        return self.download_pdf().extract_amharic_text().save_to_jsonl()


In [ ]:
save_dir = Path("/content/drive/MyDrive/scraped_pdfs")
file_name = "amharic_output.jsonl"

Pdf_urls = [
    "https://laws.moj.gov.et/sites/default/files/registry/cae0761a-ef52-48c9-b992-8364128730ec.pdf",
    "https://mols.gov.et/wp-content/uploads/2022/07/Labour-proclamation-1156.pdf",
    "https://fjli.gov.et/researches/Research42.pdf",
    "https://nbe.gov.et/wp-content/uploads/2023/04/Reinsurance-Company-Establishment.pdf",
    "https://fjli.gov.et/researches/Research23.pdf",
    "https://fjli.gov.et/researches/Research27.pdf"
]

for pdf_url in Pdf_urls:
    try:
        scraper = AmharicPDFFetcher(pdf_url, save_dir, file_name)
        scraper.run()
    except Exception as e:
        print("Failed to download file:", pdf_url)
        pass


✅ PDF downloaded to /content/drive/MyDrive/scraped_pdfs/temp.pdf
✅ Extracted 477 Amharic lines.
✅ Saved to /content/drive/MyDrive/scraped_pdfs/amharic_output.jsonl
✅ PDF downloaded to /content/drive/MyDrive/scraped_pdfs/temp.pdf
✅ Extracted 0 Amharic lines.
✅ Saved to /content/drive/MyDrive/scraped_pdfs/amharic_output.jsonl
Failed to download file: https://fjli.gov.et/researches/Research42.pdf
✅ PDF downloaded to /content/drive/MyDrive/scraped_pdfs/temp.pdf
✅ Extracted 0 Amharic lines.
✅ Saved to /content/drive/MyDrive/scraped_pdfs/amharic_output.jsonl
Failed to download file: https://fjli.gov.et/researches/Research23.pdf
Failed to download file: https://fjli.gov.et/researches/Research27.pdf
